In [ ]:
pip install Sastrawi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Download Sastrawi stopwords
stopword_factory = StopWordRemoverFactory()
stopwords_id = stopword_factory.get_stop_words()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset tubes/Dataset_kesehatan_mental.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Question_ID  99 non-null     int64 
 1   Questions    99 non-null     object
 2   Answers      99 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [ ]:
df.head()

,Question_ID,Questions,Answers
0,3875305,Apa yang dimaksud dengan kesehatan mental?,Kesehatan mental adalah suatu kondisi seseoran...
1,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...
2,2110618,Siapa yang terpengaruh oleh penyakit mental?,Gangguan kesehatan mental dapat dialami oleh s...
3,6361820,Apa yang menyebabkan penyakit mental?,Penyakit mental dapat disebabkan oleh berbagai...
4,9434130,Apa saja tanda-tanda peringatan penyakit mental?,Beberapa tanda peringatan gangguan kesehatan m...


In [ ]:
df = df.drop(["Question_ID"], axis="columns")

In [ ]:
# Preprocessing functions
def preprocess_text(text):
    text = text.lower() #Mengubah seluruh teks menjadi huruf kecil
    text = re.sub(r'[^\w\s]', '', text) #Menghapus karakter non-alfanumerik dan tanda baca
    words = word_tokenize(text) #Tokenisasi teks menggunakan word_tokenize dari NLTK
    wnl = WordNetLemmatizer()
    text = " ".join([wnl.lemmatize(t) for t in words])
    text = " ".join(t for t in text.split() if t not in stopwords_id)
    return text

In [ ]:
# menerapkan fungsi preprocess_text pada "Questions"
df["Questions"] = df["Questions"].apply(preprocess_text)

preprocessed_questions = df[["Questions"]]

# Menampilkan "Questions" yang telah di preprocessing
display(preprocessed_questions)

,Questions
0,apa dimaksud kesehatan mental
1,apa dimaksud penyakit mental
2,siapa terpengaruh penyakit mental
3,apa menyebabkan penyakit mental
4,apa tandatanda peringatan penyakit mental
...,...
94,bagaimana tahu minum alkohol terlalu banyak
95,cannabis berbahaya dilegalkan
96,bagaimana cara meyakinkan anakanak menggunakan...
97,apa status hukum bukti cbd oil


In [ ]:
# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df["Questions"])

In [ ]:
# Create a label encoder for the answers
label_encoder = LabelEncoder()
df["Answers_Code"] = label_encoder.fit_transform(df["Answers"])

In [ ]:
display(df)

,Questions,Answers,Answers_Code
0,apa dimaksud kesehatan mental,Kesehatan mental adalah suatu kondisi seseoran...,36
1,apa dimaksud penyakit mental,Penyakit mental adalah kondisi kesehatan yang ...,57
2,siapa terpengaruh penyakit mental,Gangguan kesehatan mental dapat dialami oleh s...,21
3,apa menyebabkan penyakit mental,Penyakit mental dapat disebabkan oleh berbagai...,58
4,apa tandatanda peringatan penyakit mental,Beberapa tanda peringatan gangguan kesehatan m...,11
...,...,...,...
94,bagaimana tahu minum alkohol terlalu banyak,Identifikasi apakah konsumsi alkohol Anda berl...,24
95,cannabis berbahaya dilegalkan,"Cannabis smoke, misalnya, mengandung racun pen...",15
96,bagaimana cara meyakinkan anakanak menggunakan...,Anda tidak bisa. Tetapi Anda dapat memengaruhi...,4
97,apa status hukum bukti cbd oil,"Cannabidiol (CBD), komponen alami cannabis, di...",14


In [ ]:
# Train Naive Bayes classifier
mn = MultinomialNB()
mn.fit(tfidf_matrix, df["Answers_Code"])

# Function to predict the answer for a user-inputted question
def predict_answer(question):
    processed_question = preprocess_text(question)
    user_input_tfidf = tfidf_vectorizer.transform([processed_question])
    predicted_code = mn.predict(user_input_tfidf)[0]
    predicted_answer = label_encoder.inverse_transform([predicted_code])[0]
    return predicted_answer

print("Hi, ada yang bisa dibantu?")

# User interaction loop
while True:
    user_question = input("Pertanyaan: ")

    if user_question.lower() == 'bye':
        print("Goodbye!")
        break

    predicted_answer = predict_answer(user_question)
    print("Jawaban:", predicted_answer)

Hi, ada yang bisa dibantu?
Pertanyaan: apa itu kesehatan mental?
Jawaban: Kesehatan mental adalah suatu kondisi seseorang yang memungkinkan berkembangnya semua aspek perkembangan, baik fisik, intelektual, 
dan emosional yang optimal serta selaras dengan perkembangan orang lain, sehingga selanjutnya mampu berinteraksi dengan 
lingkungan sekitarnya.
Pertanyaan: siapa saja yang terpengaruh oleh penyakit mental?
Jawaban: Gangguan kesehatan mental dapat dialami oleh siapa saja, baik laki-laki maupun perempuan, muda maupun tua. Faktor yang dapat memicu 
terjadinya gangguan atau penyakit mental antara lain peristiwa traumatis, stres berat jangka panjang, riwayat penyakit mental pada pasien 
dan keluarganya, serta obat-obatan dan suplemen yang pernah atau sedang dikonsumsi.
Pertanyaan: apa aja tanda-tanda penyakit mental?
Jawaban: Beberapa tanda peringatan gangguan kesehatan mental antara lain perubahan suasana hati yang drastis dan tiba-tiba (merasa sedih atau 
marah tanpa alasan yang jelas),